## **Section 1 - Taking & Labelling the Images**

Using a USB Camera and Laptop, 1000 images were taken in batches of 5 or 10 at a time.
Changes in the image frames were made by varying

*   Object Orientation
*   Lighting
*   Object Position
*   Object Distance away from the camera
*   and the objects being in and out of the frame

Images taken were sized 640x320 (2:1) to rereduce memory usage especially while training the model later in this notebook

After images were obtained, they were labeled using [labelImg](https://github.com/tzutalin/labelImg) a graphical python tool for annorating images. 

The object classes used for labelling are:
1.   potatoe_fries
2.   chicken
3.   empty_plate
4.   ketchup_portion
5.   ketchup_bottle

The images were then randomized and split into an 80:20 ratio for training and validation/evaluating the model.
The images and labels/annotation files can be found [HERE](https://drive.google.com/drive/folders/1-PQKmn6JBVB2q4mw21Muee8vS3BoCkYE?usp=sharing)

## **Section 2 - Preparation**


Installing the required packages, including the Model Maker package from the [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) and the pycocotools library to be used for model evaluation.

Importing the installed packages

In [1]:
!apt-get update
!python -m pip install --upgrade pip

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [870 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1139 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [30.3 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1773 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2192 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [1216 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1155 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-backports/universe amd64 Packages [26.0 kB]
Get:13 http://archive

In [3]:
!pip install numba
!pip install -q tflite-model-maker
!pip install -q tflite-support
!pip install pycocotools

Import the required packages.

In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## **Section 3 - Training the Model**

### Step 1: Load the dataset

* Images in `train_data` is used to train the object detection model.
* Images in `val_data` is used to evaluate and check if the model can generalize/predict well to new images that it hasn't seen before.

In [3]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/notebooks/train',
    '/notebooks/train',
    ['chicken','ketchup_portion', 'ketchup_bottle', 'potatoe_fries', 'empty_plate']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/notebooks/validate',
    '/notebooks/validate',
    ['chicken','ketchup_portion', 'ketchup_bottle', 'potatoe_fries', 'empty_plate']
)

In [5]:
len(train_data)

800

In [6]:
len(val_data)

200

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

The performance of each EfficientDet-Lite model compared to others is shown in the table below.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>


## **Architecture: EfficientDet-Lite0**

In [5]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 50`, which means it will go through the training dataset 50 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 5` which takes 200 steps to go through the 800 images in the training dataset for each epoch.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy.

In [11]:
model = object_detector.create(train_data, model_spec=spec, batch_size=10, train_whole_model=True, epochs=100, validation_data=val_data)

Epoch 1/100


2022-04-27 10:25:23.754076: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302


80/80 [==============================] - 121s 694ms/step - det_loss: 1.3265 - cls_loss: 0.8403 - box_loss: 0.0097 - reg_l2_loss: 0.0635 - loss: 1.3900 - learning_rate: 0.0102 - gradient_norm: 1.4084 - val_det_loss: 0.7505 - val_cls_loss: 0.4660 - val_box_loss: 0.0057 - val_reg_l2_loss: 0.0636 - val_loss: 0.8141
Epoch 2/100
80/80 [==============================] - 49s 611ms/step - det_loss: 0.6002 - cls_loss: 0.3816 - box_loss: 0.0044 - reg_l2_loss: 0.0636 - loss: 0.6638 - learning_rate: 0.0125 - gradient_norm: 1.8670 - val_det_loss: 0.4720 - val_cls_loss: 0.2771 - val_box_loss: 0.0039 - val_reg_l2_loss: 0.0636 - val_loss: 0.5356
Epoch 3/100
80/80 [==============================] - 49s 610ms/step - det_loss: 0.4118 - cls_loss: 0.2742 - box_loss: 0.0028 - reg_l2_loss: 0.0636 - loss: 0.4754 - learning_rate: 0.0125 - gradient_norm: 1.7195 - val_det_loss: 0.3284 - val_cls_loss: 0.2095 - val_box_loss: 0.0024 - val_reg_l2_loss: 0.0636 - val_loss: 0.3921
Epoch 4/100
80/80 [====================

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, the 200 images in the validation dataset to is used evaluate how the model performs against new data it has never seen before.

In [12]:
model.evaluate(val_data)

4/4 [==============================] - 34s 3s/step



{'AP': 0.9485948,
 'AP50': 0.99998766,
 'AP75': 0.99685,
 'APs': 0.6,
 'APm': 0.9517487,
 'APl': 0.9475975,
 'ARmax1': 0.6342972,
 'ARmax10': 0.9752321,
 'ARmax100': 0.97695434,
 'ARs': 0.8,
 'ARm': 0.9773855,
 'ARl': 0.98198515,
 'AP_/chicken': 0.9431832,
 'AP_/ketchup_portion': 0.950431,
 'AP_/ketchup_bottle': 0.94172096,
 'AP_/potatoe_fries': 0.95777166,
 'AP_/empty_plate': 0.9498673}

### Step 5: Export Model as a TensorFlow Lite model.

In [13]:
save_path = '/notebooks/models/model_final_001.tflite' # batch = 10; epochs = 100 architecture = efficientdet_lite0

In [16]:
model.export(export_dir='.', tflite_filename=save_path)

2022-04-27 12:01:47.599926: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2022-04-27 12:02:07.165753: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-27 12:02:07.165815: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-27 12:02:07.165824: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-04-27 12:02:07.166093: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp8gk8g9pe
2022-04-27 12:02:07.295198: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-04-27 12:02:07.295270: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/tmp8gk8g9pe
2022-04-27 1

### Step 6:  Evaluate the TensorFlow Lite model.

In [17]:
model.evaluate_tflite(save_path, val_data)

200/200 [==============================] - 53s 262ms/step



{'AP': 0.9257386,
 'AP50': 0.9999938,
 'AP75': 0.99405324,
 'APs': 0.5,
 'APm': 0.92787063,
 'APl': 0.91948605,
 'ARmax1': 0.62671506,
 'ARmax10': 0.9444964,
 'ARmax100': 0.9444964,
 'ARs': 0.5,
 'ARm': 0.9476396,
 'ARl': 0.940411,
 'AP_/chicken': 0.92176294,
 'AP_/ketchup_portion': 0.9301149,
 'AP_/ketchup_bottle': 0.9151669,
 'AP_/potatoe_fries': 0.93427163,
 'AP_/empty_plate': 0.9273764}